# Generate Synthetic Data for Treasury Analytics

1. Transactions Table (Largest Dataset, 10000 records)
Reasoning: Financial transactions typically occur frequently and continuously, resulting in a large volume of records. This dataset would naturally be the largest, as it captures daily operational expenses, revenues, vendor payments, and other financial activities. Transactional data often accumulates rapidly, making it the most extensive dataset.

2. Investment Portfolio Holdings (Second Largest, 250 records)
Reasoning: Investment portfolios usually contain numerous securities, each with multiple transactions (buys, sells, dividends, interest payments). While not as frequent as daily operational transactions, investment holdings still generate substantial data, especially if historical records and periodic valuations are included.

3. External Vendor Information (~200 records)
Reasoning: This dimension table focuses on vendors with formal contracts or significant financial relationships with the organization.


4. Unclaimed Property Records (~200 records)
Reasoning: Unclaimed property records accumulate steadily but at a slower rate compared to financial transactions or investment activities. These records are typically updated periodically (monthly, quarterly, or annually), resulting in fewer records than the first two datasets.

5. Program Performance Metrics (Smallest Dataset, 20-50 records)
Reasoning: Program performance metrics are usually aggregated data points collected periodically (monthly, quarterly, or annually). They summarize program outcomes, participant counts, and costs, resulting in fewer records compared to transactional or investment data.
Recommended Dataset Sizes (for your dummy data project). 

In [148]:
import pandas as pd
import numpy as np
from faker import Faker
import random

fake = Faker()

# Set seed for reproducibility
np.random.seed(91942)
random_state = random.seed(91942)

## Generate Dataset 1: ISTO Financial Transactions
Purpose: Analyze spending patterns, identify cost-saving opportunities, and visualize financial trends.
Table Type: Fact table

**Schema:**
- Transaction_ID (unique identifier) VARCHAR
- Transaction_Date DATETIME
- Transaction_Amount FLOAT
- Transaction_Type (Expense, Revenue) VARCHAR
- Department (e.g., Operations, Investments, Community Programs) VARCHAR
- Vendor_Name VARCHAR
- Description VARCHAR

In [149]:
# Number of rows
num_rows = 10000
num_unique_vendors = 1159 

# Generate Transaction IDs (formatted as TXN00000001)
transaction_ids = [f'TXN{str(i).zfill(8)}' for i in range(1, num_rows + 1)]

# Generate random dates within a realistic range (e.g., past 2 years)
transaction_dates = [fake.date_between(start_date='-2y', end_date='today') for _ in range(num_rows)]

# Generate transaction amounts (expenses and revenues)
transaction_amounts = np.round(np.random.uniform(50, 150000, num_rows), 2)

# Randomly assign transaction types
transaction_types = np.random.choice(['Expense', 'Revenue'], size=num_rows, p=[0.7, 0.3])

# Departments
departments = ['Operations', 'Investments', 'Community Programs', 'Administration', 'IT', 'Compliance']
department_choices = np.random.choice(departments, size=num_rows)

# Vendor names (using Faker), limiting to lower unique vendors
unique_vendors = [fake.unique.company() for _ in range(num_unique_vendors)]

# Create a realistic distribution of vendor frequencies
vendor_probabilities = np.random.zipf(a=2, size=num_unique_vendors)
vendor_probabilities = vendor_probabilities / vendor_probabilities.sum()

# Assign vendors to transactions based on probabilities
vendor_names = np.random.choice(unique_vendors, size=num_rows, p=vendor_probabilities)

# Transaction descriptions (simple examples)
descriptions_expense = ['Office Supplies', 'Consulting Fees', 'Software Subscription', 'Event Sponsorship', 'Maintenance', 'Travel Expenses']
descriptions_revenue = ['Interest Income', 'Investment Returns', 'Program Fees', 'Grant Funding', 'Reimbursement']

descriptions = [
    random.choice(descriptions_expense) if t == 'Expense' else random.choice(descriptions_revenue)
    for t in transaction_types
]

# Assemble DataFrame
transactions_df = pd.DataFrame({
    'Transaction_ID': transaction_ids,
    'Transaction_Date': transaction_dates,
    'Transaction_Amount': transaction_amounts,
    'Transaction_Type': transaction_types,
    'Department': department_choices,
    'Vendor_Name': vendor_names,
    'Description': descriptions
})

# Preview the dataset
print(transactions_df.head())
print("Total transactions:", len(transactions_df))
print("Unique vendors:", transactions_df.Vendor_Name.nunique())
print("Vendors appearing more than once:", (transactions_df.Vendor_Name.value_counts() > 1).sum())


  Transaction_ID Transaction_Date  Transaction_Amount Transaction_Type  \
0    TXN00000001       2023-11-17            57080.47          Expense   
1    TXN00000002       2024-07-09           100948.94          Expense   
2    TXN00000003       2024-11-29            82489.87          Expense   
3    TXN00000004       2024-02-18           149280.56          Revenue   
4    TXN00000005       2024-07-18            34606.19          Expense   

    Department                    Vendor_Name      Description  
0  Investments  Holloway, Archer and Espinoza      Maintenance  
1   Compliance                      Green Inc  Office Supplies  
2   Operations                   Schwartz Inc  Travel Expenses  
3  Investments                Rodriguez Group    Grant Funding  
4           IT    Smith, Parsons and Williams  Office Supplies  
Total transactions: 10000
Unique vendors: 954
Vendors appearing more than once: 667


This has created a perfect dataframe with no errors or duplicates. This is unrealistic, so let's shake it up.

1. Introduce dupes
2. Add incorrect data types
3. Add logical inconsistencies
4. Give it some outliers!
5. Mess up the vendor names including INC
6. Remove a few IDs
7. Spread some NaN love throughout

In [150]:
import pandas as pd
import numpy as np
import copy

np.random.seed(91942)
random_state = random.seed(91942)
random_state_alt = random.seed(24919)

transactions_clean_df = copy.deepcopy(transactions_df)

# 1. Introduce duplicate rows
num_duplicates = int(len(transactions_df) * 0.02)  # 2% duplicates
duplicates_df = transactions_df.sample(n=num_duplicates, random_state=random_state)
transactions_df = pd.concat([transactions_df, duplicates_df], ignore_index=True)

# 2. Introduce incorrect data types
num_type_errors = int(len(transactions_df) * 0.02)  # 2% type errors
type_error_indices = transactions_df.sample(n=num_type_errors, random_state=random_state).index
transactions_df.loc[type_error_indices, 'Transaction_Amount'] = transactions_df.loc[type_error_indices, 'Transaction_Amount'].astype(str)

date_error_indices = transactions_df.sample(n=num_type_errors, random_state=random_state_alt).index
transactions_df.loc[date_error_indices, 'Transaction_Date'] = transactions_df.loc[date_error_indices, 'Transaction_Date'].astype(str) + ' INVALID'

# 3. Introduce logical inconsistencies
expense_indices = transactions_df[transactions_df['Transaction_Type'] == 'Expense'].sample(n=num_type_errors, random_state=random_state).index
transactions_df.loc[expense_indices, 'Transaction_Amount'] *= -1  # Negative amounts for expenses

invalid_department_indices = transactions_df.sample(n=num_type_errors, random_state=random_state_alt).index
transactions_df.loc[invalid_department_indices, 'Department'] = 'InvalidDept'

# 4. Introduce outliers
# Ensure 'Transaction_Amount' is numeric
transactions_df['Transaction_Amount'] = pd.to_numeric(transactions_df['Transaction_Amount'], errors='coerce')
outlier_indices = transactions_df.sample(n=num_type_errors, random_state=random_state).index

# Generate outliers using a log-normal distribution
outlier_values = np.random.lognormal(mean=10, sigma=2, size=num_type_errors)
transactions_df.loc[outlier_indices, 'Transaction_Amount'] = outlier_values

future_date_indices = transactions_df.sample(n=num_type_errors, random_state=random_state_alt).index
transactions_df.loc[future_date_indices, 'Transaction_Date'] = pd.Timestamp('today') + pd.to_timedelta(np.random.randint(30, 365, size=num_type_errors), unit='d')

# 5. Introduce inconsistent vendor names
vendor_indices = transactions_df.sample(n=num_type_errors, random_state=random_state).index
transactions_df.loc[vendor_indices, 'Vendor_Name'] = transactions_df.loc[vendor_indices, 'Vendor_Name'].apply(
    lambda x: x.replace('Inc.', 'Incorporated') if 'Inc.' in x else x + ' Inc.'
)

# 6. Introduce missing or invalid Transaction IDs
missing_id_indices = transactions_df.sample(n=num_type_errors, random_state=random_state).index
transactions_df.loc[missing_id_indices, 'Transaction_ID'] = np.nan

# 7. Introduce missing values (nulls) - needs to be last so it doesn't break the multiplication code
num_nulls = int(len(transactions_df) * 0.03)  # 3% nulls
null_indices_amount = transactions_df.sample(n=num_nulls, random_state=random_state).index
null_indices_department = transactions_df.sample(n=num_nulls, random_state=random_state_alt).index
transactions_df.loc[null_indices_amount, 'Transaction_Amount'] = np.nan
transactions_df.loc[null_indices_department, 'Department'] = np.nan

# Shuffle the DataFrame to mix errors naturally
transactions_df = transactions_df.sample(frac=1, random_state=random_state).reset_index(drop=True)

# Check the resulting DataFrame
print(transactions_df.head())
# print(transactions_df.info())


  Transaction_ID            Transaction_Date  Transaction_Amount  \
0    TXN00003763                  2023-11-19            49388.52   
1    TXN00007941  2025-09-28 15:42:22.372415            52924.48   
2    TXN00007395                  2024-02-08           140902.59   
3    TXN00009310                  2024-01-09            60473.14   
4    TXN00009084                  2025-03-10           137488.96   

  Transaction_Type      Department                    Vendor_Name  \
0          Expense  Administration  Holloway, Archer and Espinoza   
1          Expense      Compliance                    Moore Group   
2          Expense  Administration          Ponce, Carr and Brown   
3          Expense      Compliance                       Diaz Inc   
4          Expense  Administration    Sanders, Henderson and Reed   

             Description  
0  Software Subscription  
1        Consulting Fees  
2        Office Supplies  
3        Office Supplies  
4      Event Sponsorship  


C:\Users\justi\AppData\Local\Temp\ipykernel_4372\2656578920.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['103538.4' '71506.15' '120225.49' '129209.59' '69280.12' '100484.86'
 '82174.83' '114616.42' '55552.2' '135823.58' '52923.34' '62042.56'
 '120506.57' '79590.86' '123237.36' '20528.35' '35164.55' '116512.55'
 '138830.93' '133164.12' '141469.84' '144046.54' '32740.77' '93226.17'
 '138094.14' '101212.61' '44310.88' '10794.66' '32747.58' '145253.56'
 '6716.69' '124463.3' '121659.2' '100058.6' '114127.4' '40838.84'
 '124203.3' '43014.97' '110460.07' '76922.37' '110888.29' '142715.75'
 '78947.16' '71146.91' '65448.27' '10842.02' '71859.01' '137943.32'
 '105404.39' '48441.65' '26752.73' '98718.03' '97671.68' '126678.32'
 '85428.61' '141451.76' '126947.52' '67966.94' '110043.76' '27480.37'
 '143568.09' '126267.27' '91074.24' '12732.57' '147709.91' '106504.73'
 '131184.36' '119501.58' '70779.29' '1446

In [151]:
print("Number of unique vendors:", len(transactions_df.Vendor_Name.unique()))
print("Number of duplicated vendors:", transactions_df.Vendor_Name.duplicated().sum())

Number of unique vendors: 1041
Number of duplicated vendors: 9159


In [152]:
print(transactions_df.columns)


Index(['Transaction_ID', 'Transaction_Date', 'Transaction_Amount',
       'Transaction_Type', 'Department', 'Vendor_Name', 'Description'],
      dtype='object')


Finally, save the data in a horrible data format: csv

In [153]:
# Save to CSV and cringe
transactions_df.to_csv('transactions.csv', index=False)

## Generate Dataset #2 Investment Portfolio Holdings
Purpose: Analyze ISTO's investment portfolio, diversification, and risk management.
Table Type: Dimension

**Schema:**
- Holding_ID VARCHAR
- Security_Name VARCHAR
- Security_Type VARCHAR
- Quantity_Held INT
- Market_Value FLOAT
- Acquisition_Date DATETIME

In [154]:
import pandas as pd
import numpy as np
from faker import Faker

fake = Faker()
np.random.seed(91942)

# Number of holdings
num_holdings = 257

# Generate Holding IDs
holding_ids = [f'HLD{str(i).zfill(5)}' for i in range(1, num_holdings + 1)]

# Generate realistic security names (e.g., company names)
security_names = [fake.unique.company() for _ in range(num_holdings)]

# Security types distribution
security_types = np.random.choice(
    ['Stock', 'Bond', 'ETF', 'Mutual Fund'],
    size=num_holdings,
    p=[0.5, 0.2, 0.2, 0.1]
)

# Quantity held (realistic quantities)
quantity_held = np.random.randint(100, 5000, size=num_holdings)

# Market value per unit (realistic prices)
market_price_per_unit = np.round(np.random.uniform(10, 500, size=num_holdings), 2)

# Calculate total market value
market_values = np.round(quantity_held * market_price_per_unit, 2)

# Acquisition dates (within past 5 years)
acquisition_dates = pd.to_datetime([fake.date_between(start_date='-5y', end_date='today') for _ in range(num_holdings)])


# Assemble DataFrame
portfolio_holdings_df = pd.DataFrame({
    'Holding_ID': holding_ids,
    'Security_Name': security_names,
    'Security_Type': security_types,
    'Quantity_Held': quantity_held,
    'Market_Price_Per_Unit': market_price_per_unit,
    'Total_Market_Value': market_values,
    'Acquisition_Date': acquisition_dates
})

# Preview the dataset
print(portfolio_holdings_df.head())


  Holding_ID                  Security_Name Security_Type  Quantity_Held  \
0   HLD00001        Rogers, Lane and Miller         Stock            544   
1   HLD00002  Jackson, Blankenship and Hart          Bond           1106   
2   HLD00003                  Hodge-Hawkins          Bond           1841   
3   HLD00004        Scott, Chapman and Buck   Mutual Fund           1427   
4   HLD00005       Rojas, Thomas and Zavala         Stock            805   

   Market_Price_Per_Unit  Total_Market_Value Acquisition_Date  
0                 493.30           268355.20       2022-09-07  
1                  44.65            49382.90       2022-11-17  
2                  53.83            99101.03       2020-10-04  
3                 424.49           605747.23       2025-01-18  
4                  96.97            78060.85       2020-07-20  


In [155]:
print(transactions_df.columns)


Index(['Transaction_ID', 'Transaction_Date', 'Transaction_Amount',
       'Transaction_Type', 'Department', 'Vendor_Name', 'Description'],
      dtype='object')


Now that we have a perfect dataset, let's make it crappy!

In [156]:
import numpy as np
import pandas as pd
from faker import Faker

fake = Faker()
np.random.seed(91942)
random_state=random.seed(91942)
random_state_alt = random.seed(24919)

# 1. Introduce duplicate holdings with slight variations (3%)
num_duplicates = int(len(portfolio_holdings_df) * 0.03)
duplicates_df = portfolio_holdings_df.sample(n=num_duplicates, random_state=random_state).copy()

duplicates_df['Security_Name'] = duplicates_df['Security_Name'].apply(
    lambda x: x + ' Inc.' if 'Inc.' not in x else x.replace('Inc.', 'Incorporated')
)
duplicates_df['Acquisition_Date'] += pd.to_timedelta(
    np.random.randint(-10, 10, size=num_duplicates), unit='d'
)

# 2. Introduce missing values in 'Market_Price_Per_Unit' (2%)
num_nulls = int(len(portfolio_holdings_df) * 0.02)
null_indices = portfolio_holdings_df.sample(n=num_nulls, random_state=random_state).index
portfolio_holdings_df.loc[null_indices, 'Market_Price_Per_Unit'] = np.nan

# 3. Introduce unrealistic market prices (outliers) (2%)
num_outliers = int(len(portfolio_holdings_df) * 0.02)
outlier_indices = portfolio_holdings_df.sample(n=num_outliers, random_state=random_state_alt).index
portfolio_holdings_df.loc[outlier_indices, 'Market_Price_Per_Unit'] *= np.random.choice([0.01, 100], size=num_outliers)
portfolio_holdings_df['Total_Market_Value'] = portfolio_holdings_df['Quantity_Held'] * portfolio_holdings_df['Market_Price_Per_Unit']

# 4. Introduce incorrect security types (misclassification) (3%)
num_misclassified = int(len(portfolio_holdings_df) * 0.03)
misclassified_indices = portfolio_holdings_df.sample(n=num_misclassified, random_state=random_state).index
portfolio_holdings_df.loc[misclassified_indices, 'Security_Type'] = np.random.choice(['Stock', 'Bond', 'ETF', 'Mutual Fund'], size=num_misclassified)

# 5. Introduce future acquisition dates (2%)
num_future_dates = int(len(portfolio_holdings_df) * 0.02)
future_indices = portfolio_holdings_df.sample(n=num_future_dates, random_state=random_state_alt).index
portfolio_holdings_df.loc[future_indices, 'Acquisition_Date'] = pd.Timestamp('today') + pd.to_timedelta(np.random.randint(30, 365, size=num_future_dates), unit='d')

# Combine duplicates back into the original DataFrame
portfolio_holdings_df = pd.concat([portfolio_holdings_df, duplicates_df], ignore_index=True)

# Shuffle the DataFrame to mix errors naturally
portfolio_holdings_df = portfolio_holdings_df.sample(frac=1, random_state=random_state).reset_index(drop=True)

# Check the resulting DataFrame
print(portfolio_holdings_df.head())


  Holding_ID             Security_Name Security_Type  Quantity_Held  \
0   HLD00144               Ramirez Ltd         Stock           2908   
1   HLD00145             Ball and Sons           ETF           2357   
2   HLD00226                 Doyle PLC         Stock           2428   
3   HLD00182  Woods, Spencer and Dixon   Mutual Fund           4352   
4   HLD00239    Cook, Hill and Salazar          Bond           4716   

   Market_Price_Per_Unit  Total_Market_Value Acquisition_Date  
0                 157.64           458417.12       2023-06-17  
1                 487.14          1148188.98       2023-02-19  
2                 201.93           490286.04       2022-10-01  
3                 492.02          2141271.04       2022-01-14  
4                 426.09          2009440.44       2022-04-08  


In [157]:
# Save to CSV and cringe some more
portfolio_holdings_df.to_csv('portfolio_holdings.csv', index=False)

# Generate Dataset #3: External Vendor Information
Contains descriptive attributes about the vendors, and provides context for analyzing transactions from the transaction table. 

Table Type: Dimension
**Schema:**
- Vendor_ID VARHCAR
- Vendor_Name VARCHAR
- Service Type VARCHAR
- Contract_Start_Date DATETIME
- Contract_End_Date DATETIME
- Amount_Paid_YTD FLOAT


Considerations: 
Generating this table has a few dependencies: 
1. The external vendors with contracts should be a subset of the vendors making transactions
2. If it is an external vendor with a contract, they should not be making transactions AFTER their contract has ended


In [158]:
import pandas as pd
import numpy as np
from faker import Faker

# Initialize Faker for generating random data
fake = Faker()
np.random.seed(91942)

# Number of vendors to include in the External Vendor Information Table
num_vendors = 200

# Uses the unique_vendors from the Transactions Table
external_vendor_subset = np.random.choice(unique_vendors, size=num_vendors, replace=False)

# Generate unique Vendor IDs for the subset
vendor_ids = [f'VND{str(i).zfill(4)}' for i in range(1, num_vendors + 1)]

# Generate random Service Types
service_types = np.random.choice(
    ['IT Services', 'Consulting', 'Supplies', 'Financial Services', 'Maintenance'],
    size=num_vendors
)

# Generate random Contract Start Dates (within the past 5 years)
contract_start_dates = pd.to_datetime(
    [fake.date_between(start_date='-5y', end_date='today') for _ in range(num_vendors)]
)

# Generate random Contract End Dates (1–3 years after the start date)
contract_end_dates = contract_start_dates + pd.to_timedelta(
    np.random.randint(365, 1095, size=num_vendors), unit='d'
)

# Simulate Transactions DataFrame to get the latest transaction date for each vendor
# Generate transaction dates within the past 2 years
transaction_dates = [fake.date_between(start_date='-2y', end_date='today') for _ in range(10000)]

# Generate a single Zipf distribution for vendor probabilities
zipf_distribution = np.random.zipf(a=2, size=1159)
vendor_probabilities = zipf_distribution / zipf_distribution.sum()

# Assign vendors to transactions based on the normalized probabilities
transaction_vendors = np.random.choice(unique_vendors, size=10000, p=vendor_probabilities)

# Create the Transactions DataFrame
transactions_vendors_df = pd.DataFrame({
    'Transaction_Date': transaction_dates,
    'Vendor_Name': transaction_vendors
})

# Ensure Contract End Dates are after the latest transaction date for each vendor
latest_transaction_dates = transactions_vendors_df.groupby('Vendor_Name')['Transaction_Date'].max().apply(pd.Timestamp)
contract_end_dates = [
    max(latest_transaction_dates.get(vendor, pd.Timestamp('today')), end_date)
    for vendor, end_date in zip(external_vendor_subset, contract_end_dates)
]

# Generate random Amount Paid Year-to-Date (YTD)
amount_paid_ytd = np.round(np.random.uniform(5000, 200000, size=num_vendors), 2)

# Create the External Vendor Information DataFrame
external_vendor_df = pd.DataFrame({
    'Vendor_ID': vendor_ids,
    'Vendor_Name': external_vendor_subset,
    'Service_Type': service_types,
    'Contract_Start_Date': contract_start_dates,
    'Contract_End_Date': contract_end_dates,
    'Amount_Paid_YTD': amount_paid_ytd
})

# Preview the table
print(external_vendor_df.head())



  Vendor_ID               Vendor_Name Service_Type Contract_Start_Date  \
0   VND0001            Holder-Roberts   Consulting          2023-10-25   
1   VND0002                 Dixon LLC     Supplies          2021-10-15   
2   VND0003  Murray, Fuller and Smith  Maintenance          2020-09-01   
3   VND0004           Whitehead Group  Maintenance          2022-02-21   
4   VND0005           Thompson-Barker     Supplies          2022-03-19   

           Contract_End_Date  Amount_Paid_YTD  
0 2025-09-30 00:00:00.000000         89484.49  
1 2025-02-01 00:00:00.000000         75736.47  
2 2025-04-17 15:42:22.835525         44447.81  
3 2024-11-04 00:00:00.000000         16022.84  
4 2025-04-17 15:42:22.835525         88657.40  


Once again, we have perfect data that we need to ruin by 
Adding duplicate rows
Inconsistent date formats
Outliers
Logical inconsistencies
Inconsistent naming conventions
missing IDs
Null values

In [159]:
import random

# Introduce Duplicate Rows
num_duplicates = int(len(external_vendor_df) * 0.02)  # 2% duplicates
duplicates_df = external_vendor_df.sample(n=num_duplicates, random_state=42)
external_vendor_df = pd.concat([external_vendor_df, duplicates_df], ignore_index=True)

# Introduce Inconsistent Data Formats
# Convert some dates to strings
num_inconsistent_dates = int(len(external_vendor_df) * 0.05)  # 5% inconsistent dates
date_format_indices = external_vendor_df.sample(n=num_inconsistent_dates, random_state=42).index
external_vendor_df.loc[date_format_indices, 'Contract_Start_Date'] = external_vendor_df.loc[
    date_format_indices, 'Contract_Start_Date'
].astype(str)

# Modify Vendor Names to Introduce Inconsistencies
vendor_name_indices = external_vendor_df.sample(n=num_inconsistent_dates, random_state=42).index
external_vendor_df.loc[vendor_name_indices, 'Vendor_Name'] = external_vendor_df.loc[
    vendor_name_indices, 'Vendor_Name'
].apply(lambda x: x.lower() if random.random() > 0.5 else x.upper())

# Introduce Outliers in Amount_Paid_YTD
outlier_indices = external_vendor_df.sample(n=num_inconsistent_dates, random_state=42).index
external_vendor_df.loc[outlier_indices, 'Amount_Paid_YTD'] = external_vendor_df.loc[
    outlier_indices, 'Amount_Paid_YTD'
] * np.random.choice([0.01, 100], size=num_inconsistent_dates)

# Introduce Logical Inconsistencies
# Set Contract_End_Date earlier than Contract_Start_Date
logical_error_indices = external_vendor_df.sample(n=num_inconsistent_dates, random_state=42).index
external_vendor_df.loc[logical_error_indices, 'Contract_End_Date'] = external_vendor_df.loc[
    logical_error_indices, 'Contract_Start_Date'
] - pd.Timedelta(days=random.randint(1, 365))

# Introduce Missing Values (Nulls) - LAST STEP
num_nulls = int(len(external_vendor_df) * 0.05)  # 5% nulls
null_indices_end_date = external_vendor_df.sample(n=num_nulls, random_state=42).index
null_indices_service_type = external_vendor_df.sample(n=num_nulls, random_state=24).index
null_indices_vendor_id = external_vendor_df.sample(n=num_nulls, random_state=36).index

external_vendor_df.loc[null_indices_end_date, 'Contract_End_Date'] = pd.NaT
external_vendor_df.loc[null_indices_service_type, 'Service_Type'] = None
external_vendor_df.loc[null_indices_vendor_id, 'Vendor_ID'] = None

# Preview the table with errors
print(external_vendor_df.head(20))


   Vendor_ID                      Vendor_Name        Service_Type  \
0    VND0001                   Holder-Roberts          Consulting   
1    VND0002                        Dixon LLC            Supplies   
2    VND0003         Murray, Fuller and Smith         Maintenance   
3    VND0004                  Whitehead Group         Maintenance   
4    VND0005                  Thompson-Barker            Supplies   
5    VND0006           Mata, Koch and Barrett         Maintenance   
6    VND0007         Pierce, Mendoza and Hill         Maintenance   
7    VND0008       Hughes, Thomas and Schmidt  Financial Services   
8    VND0009                       Romero LLC         Maintenance   
9    VND0010                 GREGORY AND SONS          Consulting   
10   VND0011                     Guerrero PLC         IT Services   
11   VND0012  Gonzalez, Stewart and Zimmerman            Supplies   
12   VND0013                   Fuller-Ramirez  Financial Services   
13   VND0014                      

In [160]:
# Optional: Save the table to a CSV file
external_vendor_df.to_csv('external_vendor_information.csv', index=False)

## Generate Dataset #4: Unclaimed Property Records
Purpose: Evaluate effectiveness of ISTO's unclaimed property program, identify trends, and visualize property types.
Table Type: Fact Table
**Schema:** 
- Property_ID VARCHAR
- Owner_Name VARCHAR
- Property_Type VARCHAR
- Reported_Date DATETIME
- Property_Value FLOAT
- Claim_Status VARCHAR

Considerations: 
1. There will be a mix of vendors and people who own unclaimed property
2. Some of the vendors will be active vendors, but some will not be active, and will not appear in any of the tables (perhaps former contract holders)
3. Dates will only be investigated quarterly; this isn't a daily investigation

In [161]:
import pandas as pd
import numpy as np
from faker import Faker

# Initialize Faker for generating random data
fake = Faker()
np.random.seed(91942)

# Number of unclaimed property records
num_records = 200

# Use active vendors from the Transactions Table
# unique_vendors = unique_vendors # from the Transactions processing
active_vendors = np.random.choice(unique_vendors, size=int(num_records * 0.1), replace=False)  # 40% active vendors

# Generate new vendors (inactive vendors)
inactive_vendors = [fake.company() for _ in range(int(num_records * 0.4))]  # 30% inactive vendors

# Generate individuals 
individuals = [fake.name() for _ in range(int(num_records * 0.5))]

# Combine active and inactive vendors
owner_names = np.concatenate([active_vendors, inactive_vendors, individuals])
np.random.shuffle(owner_names)  # Shuffle to mix active and inactive vendors

# Generate random Owner Types (Individual or Vendor)
owner_types = ['Vendor' if owner in active_vendors or owner in inactive_vendors else "Individual" for owner in owner_names]

# Generate random Property Types
property_types = np.random.choice(
    ['Bank Account', 'Insurance Claim', 'Stocks', 'Safe Deposit Box', 'Uncashed Check', 'Bonds'],
    size=num_records
)

# Generate random Reported Dates (limited to quarterly intervals in the past 5 years)
quarters = pd.date_range(start='2018-01-01', end='2023-12-31', freq='Q')
reported_dates = np.random.choice(quarters, size=num_records)

# Generate random Property Values
property_values = np.round(np.random.uniform(100, 10000, size=num_records), 2)

# Generate random Claim Status
claim_status = np.random.choice(['Claimed', 'Unclaimed'], size=num_records, p=[0.3, 0.7])

# Create the Unclaimed Property Records DataFrame
unclaimed_property_df = pd.DataFrame({
    'Property_ID': [f'UP{str(i).zfill(4)}' for i in range(1, num_records + 1)],
    'Owner_Type': owner_types,
    'Owner_Name': owner_names,
    'Property_Type': property_types,
    'Reported_Date': reported_dates,
    'Property_Value': property_values,
    'Claim_Status': claim_status
})

# Preview the table
print(unclaimed_property_df.head())



  Property_ID  Owner_Type                     Owner_Name     Property_Type  \
0      UP0001      Vendor  Rodriguez, Nguyen and Elliott  Safe Deposit Box   
1      UP0002  Individual                  Cynthia Baker      Bank Account   
2      UP0003  Individual                  Andrew Barnes   Insurance Claim   
3      UP0004  Individual                Jennifer Briggs  Safe Deposit Box   
4      UP0005  Individual                  Krista Spence   Insurance Claim   

  Reported_Date  Property_Value Claim_Status  
0    2018-09-30         5011.73    Unclaimed  
1    2022-03-31         8860.79    Unclaimed  
2    2022-03-31         6253.99    Unclaimed  
3    2018-12-31         7916.36    Unclaimed  
4    2018-12-31          443.81    Unclaimed  


C:\Users\justi\AppData\Local\Temp\ipykernel_4372\3275233515.py:36: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  quarters = pd.date_range(start='2018-01-01', end='2023-12-31', freq='Q')


Oooooh, nice data again. Let's rain on this parade with some duplicates, inconsistencies, outliers and nulls! 

In [162]:
import random

# Introduce Duplicate Records
num_duplicates = int(len(unclaimed_property_df) * 0.02)  # 2% duplicates
duplicates_df = unclaimed_property_df.sample(n=num_duplicates, random_state=random_state)
unclaimed_property_df = pd.concat([unclaimed_property_df, duplicates_df], ignore_index=True)

# Introduce Inconsistent Formats
# Convert some dates to strings
num_inconsistent_dates = int(len(unclaimed_property_df) * 0.01)  # 1% inconsistent dates
date_format_indices = unclaimed_property_df.sample(n=num_inconsistent_dates, random_state=random_state).index
unclaimed_property_df.loc[date_format_indices, 'Reported_Date'] = unclaimed_property_df.loc[
    date_format_indices, 'Reported_Date'
].astype(str)

# Modify Owner Names to Introduce Inconsistencies
owner_name_indices = unclaimed_property_df.sample(n=num_inconsistent_dates, random_state=random_state).index
unclaimed_property_df.loc[owner_name_indices, 'Owner_Name'] = unclaimed_property_df.loc[
    owner_name_indices, 'Owner_Name'
].apply(lambda x: x.lower() if isinstance(x, str) and random.random() > 0.5 else x.upper() if isinstance(x, str) else x)

# Introduce Logical Inconsistencies
# Set Claim_Status to "Claimed" but leave Owner_Name as null
logical_error_indices = unclaimed_property_df.sample(n=num_inconsistent_dates, random_state=random_state).index
unclaimed_property_df.loc[logical_error_indices, 'Claim_Status'] = 'Claimed'
unclaimed_property_df.loc[logical_error_indices, 'Owner_Name'] = None

# Introduce Outliers in Property_Value
outlier_indices = unclaimed_property_df.sample(n=num_inconsistent_dates, random_state=random_state).index
unclaimed_property_df.loc[outlier_indices, 'Property_Value'] = unclaimed_property_df.loc[
    outlier_indices, 'Property_Value'
] * np.random.choice([0.01, 1000], size=num_inconsistent_dates)

# Introduce Missing Values (Nulls) - LAST STEP
num_nulls = int(len(unclaimed_property_df) * 0.01)  # 1% nulls
null_indices_owner_name = unclaimed_property_df.sample(n=num_nulls, random_state=random_state).index
null_indices_property_type = unclaimed_property_df.sample(n=num_nulls, random_state=random_state_alt).index

unclaimed_property_df.loc[null_indices_owner_name, 'Owner_Name'] = None
unclaimed_property_df.loc[null_indices_property_type, 'Property_Type'] = None

# Preview the table with errors
print(unclaimed_property_df.head(20))


   Property_ID  Owner_Type                     Owner_Name     Property_Type  \
0       UP0001      Vendor  Rodriguez, Nguyen and Elliott  Safe Deposit Box   
1       UP0002  Individual                  Cynthia Baker      Bank Account   
2       UP0003  Individual                  Andrew Barnes   Insurance Claim   
3       UP0004  Individual                Jennifer Briggs  Safe Deposit Box   
4       UP0005  Individual                  Krista Spence   Insurance Claim   
5       UP0006      Vendor                  Johnson Group   Insurance Claim   
6       UP0007      Vendor                   Burke-Vargas    Uncashed Check   
7       UP0008  Individual                  David Calhoun    Uncashed Check   
8       UP0009      Vendor                      White Ltd             Bonds   
9       UP0010      Vendor                  Maldonado LLC    Uncashed Check   
10      UP0011  Individual                   Kylie Larsen    Uncashed Check   
11      UP0012      Vendor                 Watts-Eri

In [163]:

# Optional: Save the table to a CSV file
unclaimed_property_df.to_csv('unclaimed_property_records.csv', index=False)

# Generate Dataset 5: Program Performance Metrics
Purpose: Evaluate effectiveness and impact of ISTO programs, identify areas for improvement.

Table Type: Dimension

**Schema:**
- Program_ID VARCHAR
- Program_Name VARCHAR
- Reporting_Period DATETIME
- Participants VARCHAR
- Successful_Outcomes VARCHAR
- Program_Cost FLOAT

Considerations: 
This table aggregates some of the data from other existing tables, so it cannot be generated without using some of the previous data, while interpolating additional information. 
1. Program Cost aggregation from Transactions
2. Participant-related transaction count
3. Vendor Costs from External Vendor Information
4. Unclaimed property such as refunds or grants not climaed
5. Investment Portfolio Holding gunding sources that can link to market value or quantity held


In [164]:
# Ensure Transaction_Date is in datetime format
transactions_clean_df['Transaction_Date'] = pd.to_datetime(transactions_clean_df['Transaction_Date'], errors='coerce')

# Drop rows with invalid or missing Transaction_Date
transactions_clean_df = transactions_clean_df.dropna(subset=['Transaction_Date'])

# Filter transactions related to programs
program_related_transactions = transactions_clean_df[
    (transactions_clean_df['Description'] == 'Program Fees') |
    (transactions_clean_df['Department'] == 'Community Programs')
]

# Aggregate program costs by reporting period (quarterly)
program_metrics = program_related_transactions.groupby(
    [pd.Grouper(key='Transaction_Date', freq='QE'), 'Description']
).agg(
    Program_Cost=('Transaction_Amount', 'sum')
).reset_index()

# Rename columns for clarity
program_metrics.rename(columns={'Description': 'Program_Name', 'Transaction_Date': 'Reporting_Period'}, inplace=True)

# Generate random participants for each program
program_metrics['Participants'] = np.random.randint(50, 1000, size=len(program_metrics))

# Generate successful outcomes as a subset of participants
program_metrics['Successful_Outcomes'] = program_metrics['Participants'].apply(lambda x: np.random.randint(int(x * 0.5), x + 1))

# Add unique Program IDs
program_metrics['Program_ID'] = [f'PRG{str(i).zfill(3)}' for i in range(1, len(program_metrics) + 1)]

# Add additional metrics

# Example budget allocations (randomized for demonstration purposes)
program_metrics['Budget_Allocation'] = np.round(program_metrics['Program_Cost'] * np.random.uniform(1.1, 1.5, size=len(program_metrics)), 2)

# Calculate Budget Utilization Rate
program_metrics['Budget_Utilization_Rate'] = (program_metrics['Program_Cost'] / program_metrics['Budget_Allocation'] * 100).round(2)

# Calculate Completion Rate
program_metrics['Completion_Rate'] = (program_metrics['Successful_Outcomes'] / program_metrics['Participants'] * 100).round(2).astype(str) + '%'

# Generate Participant Satisfaction scores (randomized between 3.5 and 5.0)
program_metrics['Participant_Satisfaction'] = [round(np.random.uniform(3.5, 5.0), 1) for _ in range(len(program_metrics))]

# Calculate Cost per Participant
program_metrics['Cost_per_Participant'] = (program_metrics['Program_Cost'] / program_metrics['Participants']).round(2)

# Calculate Cost per Successful Outcome
program_metrics['Cost_per_Successful_Outcome'] = (program_metrics['Program_Cost'] / program_metrics['Successful_Outcomes']).round(2)

# Add On-Time Completion status (randomized for demonstration purposes)
program_metrics['On_Time_Completion'] = np.random.choice(['Yes', 'No'], size=len(program_metrics))

# Preview the updated Program Performance Metrics Table
print(program_metrics.head())



  Reporting_Period        Program_Name  Program_Cost  Participants  \
0       2023-06-30     Consulting Fees    1038370.83           445   
1       2023-06-30   Event Sponsorship    1236303.87           692   
2       2023-06-30       Grant Funding     600633.30           243   
3       2023-06-30     Interest Income     629410.32           329   
4       2023-06-30  Investment Returns     653405.84           910   

   Successful_Outcomes Program_ID  Budget_Allocation  Budget_Utilization_Rate  \
0                  268     PRG001         1231197.25                    84.34   
1                  549     PRG002         1740815.19                    71.02   
2                  171     PRG003          769051.49                    78.10   
3                  187     PRG004          760824.21                    82.73   
4                  561     PRG005          910702.12                    71.75   

  Completion_Rate  Participant_Satisfaction  Cost_per_Participant  \
0          60.22%      

Lovely. Guess what we're going to do with this masterpiece? 

In [165]:
import random

# 1. Introduce Logical Inconsistencies
# Set Successful_Outcomes greater than Participants
logical_error_indices = program_metrics.sample(n=num_nulls, random_state=random_state).index
program_metrics.loc[logical_error_indices, 'Successful_Outcomes'] = program_metrics.loc[
    logical_error_indices, 'Participants'
].apply(lambda x: x + random.randint(1, 50) if x is not None else None)

# Set Budget_Utilization_Rate above 100% or below 0%
budget_error_indices = program_metrics.sample(n=num_nulls, random_state=random_state_alt).index
program_metrics.loc[budget_error_indices, 'Budget_Utilization_Rate'] = program_metrics.loc[
    budget_error_indices, 'Budget_Utilization_Rate'
].apply(lambda x: x * random.choice([-1, 2]))

# Assign On_Time_Completion as "Yes" for programs with no participants
completion_error_indices = program_metrics[program_metrics['Participants'].isnull()].index
program_metrics.loc[completion_error_indices, 'On_Time_Completion'] = 'Yes'

# 2. Introduce Outliers
# Add extreme values to Program_Cost
outlier_indices = program_metrics.sample(n=num_nulls, random_state=random_state).index
program_metrics.loc[outlier_indices, 'Program_Cost'] = program_metrics.loc[
    outlier_indices, 'Program_Cost'
] * random.choice([10, 0.01])

# 3. Introduce Duplicate Rows
num_duplicates = int(len(program_metrics) * 0.02)  # 2% duplicates
duplicates_df = program_metrics.sample(n=num_duplicates, random_state=random_state)
program_metrics = pd.concat([program_metrics, duplicates_df], ignore_index=True)

# 4. Introduce Inconsistent Formats
# Convert some Reporting_Period values to strings
format_error_indices = program_metrics.sample(n=num_nulls, random_state=random_state).index
program_metrics.loc[format_error_indices, 'Reporting_Period'] = program_metrics.loc[
    format_error_indices, 'Reporting_Period'
].astype(str) + ' INVALID'

# Modify Program_Name to introduce inconsistent capitalization
name_error_indices = program_metrics.sample(n=num_nulls, random_state=random_state).index
program_metrics.loc[name_error_indices, 'Program_Name'] = program_metrics.loc[
    name_error_indices, 'Program_Name'
].apply(lambda x: x.lower() if random.random() > 0.5 else x.upper())

# 5. Introduce Incorrect Data Types
# Convert numeric columns to strings
type_error_indices = program_metrics.sample(n=num_nulls, random_state=random_state).index
program_metrics.loc[type_error_indices, 'Program_Cost'] = program_metrics.loc[
    type_error_indices, 'Program_Cost'
].astype(str)

# 6. Introduce Missing Values (Nulls)
num_nulls = int(len(program_metrics) * 0.05)  # 5% nulls
null_indices_participants = program_metrics.sample(n=num_nulls, random_state=random_state).index
null_indices_budget = program_metrics.sample(n=num_nulls, random_state=random_state_alt).index

program_metrics.loc[null_indices_participants, 'Participants'] = None
program_metrics.loc[null_indices_budget, 'Budget_Allocation'] = None

# Shuffle the DataFrame to mix errors naturally
program_metrics = program_metrics.sample(frac=1, random_state=random_state).reset_index(drop=True)

# Preview the table with errors
print(program_metrics.head(20))


       Reporting_Period           Program_Name Program_Cost  Participants  \
0   2025-06-30 00:00:00     Investment Returns    113747.08         588.0   
1   2024-09-30 00:00:00        Office Supplies   2665792.66         884.0   
2   2023-12-31 00:00:00  Software Subscription   1753907.94         920.0   
3   2024-09-30 00:00:00     Investment Returns    830233.31         583.0   
4   2024-12-31 00:00:00        Travel Expenses   2466356.86         704.0   
5   2024-12-31 00:00:00  Software Subscription   1484993.45         878.0   
6    2024-12-31 INVALID     Investment Returns   1228261.45         783.0   
7   2024-03-31 00:00:00        Interest Income     725662.6         514.0   
8   2023-12-31 00:00:00            Maintenance   2186279.41         546.0   
9   2025-06-30 00:00:00        Travel Expenses    952662.71         591.0   
10  2025-06-30 00:00:00  Software Subscription    333186.88         231.0   
11  2024-03-31 00:00:00      Event Sponsorship   1952318.22         966.0   

C:\Users\justi\AppData\Local\Temp\ipykernel_4372\2675814484.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['2024-12-31 INVALID' '2025-03-31 INVALID']' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  program_metrics.loc[format_error_indices, 'Reporting_Period'] = program_metrics.loc[
C:\Users\justi\AppData\Local\Temp\ipykernel_4372\2675814484.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['1252925.62' '4363156.86']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  program_metrics.loc[type_error_indices, 'Program_Cost'] = program_metrics.loc[


In [166]:
# Save to excel, frown, cry... just let it all out.
program_metrics.to_excel('program_performance_metrics.xlsx', index=False, sheet_name='Program Metrics')

# Confirm the file was saved in such a horrid manner
print("Program Performance Metrics Table has been mutilated and regurgitated as 'program_performance_metrics.xlsx'.")


Program Performance Metrics Table has been mutilated and regurgitated as 'program_performance_metrics.xlsx'.


Alright, alright, alright! 
The last step is to just verify that we have all of these tables saved in the crap

In [172]:
import pandas as pd

# File paths
files = {
    "External Vendor Information": "external_vendor_information.csv"
    , "Portfolio Holdings": "portfolio_holdings.csv"
    , "Transactions": "transactions.csv"
    , "Unclaimed Property Records": "unclaimed_property_records.csv"
    , "Program Performance Metrics": "program_performance_metrics.xlsx"
}

# Loop through each file, read it, and print the head
for table_name, file_path in files.items():
    print(f"Verifying {table_name} ({file_path}):")
    try:
        # Check file extension to determine how to read the file
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        else:
            print(f"Unsupported file format for {file_path}. Skipping...\n")
            continue
        
        # Print the first few rows of the DataFrame
        if not df.empty:
            print("This crappy table exists. Lucky you.", "\n")
        else:
            print(f"The file {file_path} is empty.\n")
    except Exception as e:
        print(f"An error was squeezed out while reading {file_path}: {e}\n")


Verifying External Vendor Information (external_vendor_information.csv):
This crappy table exists. Lucky you. 

Verifying Portfolio Holdings (portfolio_holdings.csv):
This crappy table exists. Lucky you. 

Verifying Transactions (transactions.csv):
This crappy table exists. Lucky you. 

Verifying Unclaimed Property Records (unclaimed_property_records.csv):
This crappy table exists. Lucky you. 

Verifying Program Performance Metrics (program_performance_metrics.xlsx):
This crappy table exists. Lucky you. 

